In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

In [2]:
data = loadmat('data/ex3data1.mat')

In [3]:
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
data['X'].shape

(5000, 400)

In [5]:
data['y'].shape

(5000, 1)

We'll use some stuff from ex2 (note that cost is regularized)

In [6]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

In [7]:
def cost(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

From exercise 3 notes, the vectorized version of the gradient is $$ \dfrac{1}{m}X^T(h_{\theta}(x) - y) $$

In [9]:
def gradient(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept grad is a term w/o regularization
    grad[0,0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

In [10]:
from scipy.optimize import minimize

In [11]:
def one_vs_all(X, y, num_labels, learning_rate):  
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert col of ones for intercept
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params+1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows,1))
        
        # min obj fxn
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
        
    return all_theta

Sanity checks:

In [12]:
rows = data['X'].shape[0]  
params = data['X'].shape[1]

In [13]:
all_theta = np.zeros((10, params + 1))

In [14]:
X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

In [15]:
theta = np.zeros(params + 1)

In [16]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])  
y_0 = np.reshape(y_0, (rows, 1))

In [17]:
X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [18]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [19]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)  
all_theta  

array([[ -3.70247923e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -2.24803603e-10,   2.31962906e-11,   0.00000000e+00],
       [ -8.96250747e-05,   0.00000000e+00,   0.00000000e+00, ...,
          7.26120888e-09,  -6.19965351e-10,   0.00000000e+00],
       [ -8.39553307e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -7.61695536e-10,   4.64917609e-11,   0.00000000e+00],
       ..., 
       [ -7.00832401e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -6.92009001e-10,   4.29241473e-11,   0.00000000e+00],
       [ -7.65187916e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -8.09503250e-10,   5.31058705e-11,   0.00000000e+00],
       [ -6.63412325e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -3.49765837e-09,   1.13668504e-10,   0.00000000e+00]])

Now predict

In [ ]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # class probs
    h = sigmoid(X * all_theta.T)
    
    # get max prob
    h_argmax = np.argmax(h, axis=1)
    
    # add 1 since arr 0-indexed
    h_argmax += 1
    
    return h_argmax